# Lab 2 — Chain-of-Thought Prompting

**Module reference:** [Module 3, §3.4](https://github.com/kunalsuri/prompt-engineering-playbook/blob/main/learn/03-patterns.md)

This lab compares **direct-answer** prompting with **Chain-of-Thought (CoT)** on multi-step arithmetic / reasoning problems.

**Hypothesis:** CoT reduces errors on problems requiring intermediate steps.

---

### Free API Options
| Provider | Free Tier | Sign Up |
|---|---|---|
| **Google Gemini** (recommended) | 15 RPM, 1M tokens/day | [aistudio.google.com/apikey](https://aistudio.google.com/apikey) |
| **Groq** | 30 RPM, 14.4K tokens/min | [console.groq.com](https://console.groq.com) |
| **OpenAI** (paid) | Pay-per-token | [platform.openai.com](https://platform.openai.com/api-keys) |

In [ ]:
#@title 🔧 Setup — Run this cell first
!pip install -q openai

import getpass, os

print("Choose your LLM provider (all work with this lab):")
print("  1. Google Gemini (FREE — recommended)")
print("  2. Groq (FREE)")
print("  3. OpenAI (paid)")
choice = input("\nEnter 1, 2, or 3: ").strip()

if choice == "1":
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")
    BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
    API_KEY = os.environ["GOOGLE_API_KEY"]
    MODEL = "gemini-2.0-flash"
elif choice == "2":
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")
    BASE_URL = "https://api.groq.com/openai/v1"
    API_KEY = os.environ["GROQ_API_KEY"]
    MODEL = "llama-3.1-8b-instant"
else:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
    BASE_URL = None
    API_KEY = os.environ["OPENAI_API_KEY"]
    MODEL = "gpt-4o-mini"

from openai import OpenAI
client_kwargs = {"api_key": API_KEY}
if BASE_URL:
    client_kwargs["base_url"] = BASE_URL
client = OpenAI(**client_kwargs)

def complete(prompt, *, system="", temperature=0.0, max_tokens=1024):
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})
    response = client.chat.completions.create(
        model=MODEL, messages=messages,
        temperature=temperature, max_tokens=max_tokens
    )
    return response.choices[0].message.content or ""

print(f"\n✅ Connected to {MODEL}")

## Test Problems & Prompt Variants

In [ ]:
PROBLEMS = [
    ("A store has 23 apples. They sell 8, then receive a shipment of 15. A customer returns 3 apples. How many apples does the store have?", "33"),
    ("A train travels at 60 km/h for 2.5 hours, then at 80 km/h for 1.5 hours. What is the total distance traveled?", "270"),
    ("Maria has $50. She buys 3 books at $8 each and 2 pens at $3 each. How much money does she have left?", "20"),
    ("A rectangular garden is 12 meters long and 8 meters wide. A path 1 meter wide surrounds the garden. What is the area of the path?", "44"),
    ("In a class of 30 students, 40% prefer math, 30% prefer science, and the rest prefer art. How many students prefer art?", "9"),
]

SYSTEM = "You are a helpful math tutor."

DIRECT_TEMPLATE = (
    "Answer the following question. Provide ONLY the final numerical answer, "
    "with no units or explanation.\n\n"
    "Question: {question}\n"
    "Answer:"
)

COT_TEMPLATE = (
    "Answer the following question. Think step by step, showing your work. "
    "After your reasoning, provide the final numerical answer on a new line "
    "prefixed with 'ANSWER: '.\n\n"
    "Question: {question}\n"
    "Solution:"
)

print(f"Loaded {len(PROBLEMS)} test problems.")

## Run Experiment

In [ ]:
import re

def extract_number(raw):
    for line in reversed(raw.strip().splitlines()):
        if "ANSWER:" in line.upper():
            parts = line.upper().split("ANSWER:")
            num_str = parts[-1].strip().rstrip(".")
            tokens = num_str.split()
            if tokens:
                return tokens[0].replace(",", "").replace("$", "")
    numbers = re.findall(r"-?\d+\.?\d*", raw)
    return numbers[-1] if numbers else raw.strip()[:20]

def answers_match(predicted, expected):
    try:
        return abs(float(predicted) - float(expected)) < 0.5
    except ValueError:
        return predicted.strip() == expected.strip()


results = {"direct": [], "cot": []}

for variant_name, template in [("direct", DIRECT_TEMPLATE), ("cot", COT_TEMPLATE)]:
    label = "Direct" if variant_name == "direct" else "CoT"
    print(f"Running {label} variant on {len(PROBLEMS)} problems...")
    for question, expected in PROBLEMS:
        prompt = template.format(question=question)
        raw = complete(prompt, system=SYSTEM, temperature=0.0)
        predicted = extract_number(raw)
        correct = answers_match(predicted, expected)
        results[variant_name].append({
            "question": question[:50] + "...",
            "expected": expected,
            "predicted": predicted,
            "correct": correct,
            "raw_snippet": raw.strip()[:100].replace("\n", " "),
        })

print("\n✅ Experiment complete!")

## Results

In [ ]:
import pandas as pd

for variant_name in ("direct", "cot"):
    label = "Direct Answer" if variant_name == "direct" else "Chain-of-Thought"
    print(f"\n--- {label} Results ---")
    rows = []
    for r in results[variant_name]:
        rows.append({
            "Problem": r["question"],
            "Expected": r["expected"],
            "Predicted": r["predicted"],
            "Correct": "✓" if r["correct"] else "✗",
        })
    display(pd.DataFrame(rows))

# Summary
d_acc = sum(1 for r in results["direct"] if r["correct"]) / len(PROBLEMS) * 100
c_acc = sum(1 for r in results["cot"] if r["correct"]) / len(PROBLEMS) * 100

print("\n" + "="*50)
summary = pd.DataFrame([{"Metric": "Accuracy", "Direct": f"{d_acc:.0f}%", "CoT": f"{c_acc:.0f}%"}])
display(summary)

print("\n📝 Takeaway: Chain-of-Thought prompting helps the model 'show its work',")
print("reducing arithmetic errors on multi-step problems.")

## Example CoT Trace

In [ ]:
# Show a full CoT reasoning trace for Problem 1
prompt = COT_TEMPLATE.format(question=PROBLEMS[0][0])
trace = complete(prompt, system=SYSTEM, temperature=0.0)
print("Question:", PROBLEMS[0][0])
print("\n--- CoT Trace ---")
print(trace)